# Geometry Analysis

In [ ]:
import os
import numpy as np
import itertools
import csv
from pymatgen.core import Structure
from pymatgen.analysis.local_env import VoronoiNN

# Define structure paths
all_structures_path = {
"/path-to-the-directory-containing-folders-initial-and-final"
}

def find_nearest_neighbors(structure):
    """Finds the 6 nearest neighbors to the 0th atom in a structure."""
    vnn = VoronoiNN()
    neighbors = vnn.get_nn_info(structure, 0)
    neighbor_list = [(neighbor['site_index'], neighbor['site'].coords, neighbor['poly_info']['solid_angle'])
                     for neighbor in neighbors]
    return sorted(neighbor_list, key=lambda x: structure[0].distance(structure[x[0]]))[:6]

def compute_relevant_distances(structure_file):
    """Computes distances for the 6 nearest neighbors of the 0th atom."""
    structure = Structure.from_file(structure_file)
    nearest_neighbors = find_nearest_neighbors(structure)
    print(nearest_neighbors)
    indices = [index for index, _, _ in nearest_neighbors]
    atom_distances = {(0, index): structure[0].distance(structure[index]) for index in indices}
    pairwise_distances = {(i, j): structure[i].distance(structure[j]) for i, j in itertools.combinations(indices, 2)}
    return {**atom_distances, **pairwise_distances}

def compute_solid_angles(structure_file):
    """Computes the solid angles of Voronoi faces for the 6 nearest neighbors."""
    structure = Structure.from_file(structure_file)
    nearest_neighbors = find_nearest_neighbors(structure)
    return {index: solid_angle for index, _, solid_angle in nearest_neighbors}

def compute_all_differences(structure_file_1, structure_file_2):
    """Computes differences in distances and solid angles between two structures."""
    distances_1, distances_2 = compute_relevant_distances(structure_file_1), compute_relevant_distances(structure_file_2)
    solid_angles_1, solid_angles_2 = compute_solid_angles(structure_file_1), compute_solid_angles(structure_file_2)
    
    distance_differences = [abs(distances_1[p] - distances_2[p]) for p in distances_1 if p in distances_2]
    solid_angle_differences = [abs(solid_angles_1[i] - solid_angles_2[i]) for i in solid_angles_1 if i in solid_angles_2]
    
    return {
        "Mean Distance Difference (Å)": np.mean(distance_differences) if distance_differences else 0,
        "Max Distance Difference (Å)": max(distance_differences) if distance_differences else 0,
        "Mean Solid Angle Difference (sr)": np.mean(solid_angle_differences) if solid_angle_differences else 0,
        "Max Solid Angle Difference (sr)": max(solid_angle_differences) if solid_angle_differences else 0
    }

# CSV Output File
output_csv = "full_path/structure_analysis_MLIP_run_system_name.csv"
#Use the same code with MLIP replaced as LI in the output_csv for comparing LI against DFT-NEB
with open(output_csv, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Structure", "Index", "Mean Distance Difference (Å)", "Max Distance Difference (Å)",
                     "Mean Solid Angle Difference (sr)", "Max Solid Angle Difference (sr)"])
    
    for structure_name, base_path in all_structures_path.items():
        for j in range(1, 8):
            dft_structure_path = os.path.join(base_path, f"0{j}/CONTCAR")
            mace_structure_path = os.path.join(base_path, f"MLIP_folder_name/neb_optimized_{j}.vasp")  # for MLIPs
            #mace_structure_path = os.path.join(base_path, f"0{j}/POSCAR") # For LI
            if os.path.exists(dft_structure_path) and os.path.exists(mace_structure_path):
                results = compute_all_differences(dft_structure_path, mace_structure_path)
                writer.writerow([structure_name, j, results["Mean Distance Difference (Å)"], results["Max Distance Difference (Å)"],
                                 results["Mean Solid Angle Difference (sr)"], results["Max Solid Angle Difference (sr)"]])
                print(f"Processed {structure_name}, Index {j}")
            else:
                print(f"Skipping {structure_name}, Index {j} (Missing Files)")

print(f"\n✅ Results saved to {output_csv}")


# Evaluate \theta

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# Load CSV files
file2 = "full-path-to-directory-containing-MLIP-csv-file/structure_analysis_MLIP_run_system_name.csv" #MLIP file 
file1 = "full-path-to-directory-containing-LI-csv-file/structure_analysis_LI_run_system_name.csv" #LI file
#plot_output_dir = "/Users/achinthyakrishna07/Downloads/migration_barrier/plots/structure_analysis_tight_run_eb_v_interpola.png"  # Replace with actual output directory
output_file = 'full-path'
# Ensure the output directory exists
#os.makedirs(plot_output_dir, exist_ok=True)

df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)

# Merge the two datasets on Structure and Index
merged_df = df1.merge(df2, on=["Structure", "Index"], suffixes=("_sys1", "_sys2"))

# Compute differences (df2 - df1)
merged_df["Diff_Mean_Dist"] = merged_df["Mean Distance Difference (Å)_sys2"] - merged_df["Mean Distance Difference (Å)_sys1"]
merged_df["Diff_Max_Dist"] = merged_df["Max Distance Difference (Å)_sys2"] - merged_df["Max Distance Difference (Å)_sys1"]
merged_df["Diff_Mean_Solid_Angle"] = merged_df["Mean Solid Angle Difference (sr)_sys2"] - merged_df["Mean Solid Angle Difference (sr)_sys1"]
merged_df["Diff_Max_Solid_Angle"] = merged_df["Max Solid Angle Difference (sr)_sys2"] - merged_df["Max Solid Angle Difference (sr)_sys1"]

# Find the maximum of these differences
merged_df["Max_Diff"] = merged_df[["Diff_Mean_Dist", "Diff_Max_Dist", "Diff_Mean_Solid_Angle", "Diff_Max_Solid_Angle"]].max(axis=1)

# Assign labels
conditions = [
    (merged_df["Max_Diff"] > 0.1),
    (merged_df["Max_Diff"] < 0.01)
]
labels = ["Bad", "Good"]
merged_df["Label"] = np.select(conditions, labels, default="Comparable")

# Save the max difference details to file
max_diff_df = merged_df[["Structure", "Index", "Max_Diff", "Label"]]
max_diff_df.to_csv(output_file, index=False)

# Separate data based on labels
good_data = merged_df[merged_df["Label"] == "Good"]["Index"]
bad_data = merged_df[merged_df["Label"] == "Bad"]["Index"]
comparable_data = merged_df[merged_df["Label"] == "Comparable"]["Index"]

# Plot histograms
fig, axes = plt.subplots(1, 3, figsize=(15, 5), sharey=True)

# Good Histogram
axes[0].hist(good_data, bins=20, color='green', alpha=0.7, edgecolor='black')
axes[0].set_title("Good")
axes[0].set_xlabel("Index")
axes[0].set_ylabel("Frequency")
#fig.savefig(os.path.join(plot_output_dir, "histogram_good.png"))

# Bad Histogram
axes[1].hist(bad_data, bins=20, color='red', alpha=0.7, edgecolor='black')
axes[1].set_title("Bad")
axes[1].set_xlabel("Index")
#fig.savefig(os.path.join(plot_output_dir, "histogram_bad.png"))

# Comparable Histogram
axes[2].hist(comparable_data, bins=20, color='blue', alpha=0.7, edgecolor='black')
axes[2].set_title("Comparable")
axes[2].set_xlabel("Index")
#fig.savefig(os.path.join(plot_output_dir, "histogram_comparable.png"))

# Adjust layout
plt.tight_layout()
plt.show()

# Print total counts
total_counts = merged_df["Label"].value_counts()
print("Total Counts:")
print(total_counts)


# 